In [1]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sn

In [2]:
raw_data = pd.read_csv(r'.\so-survey-2017\survey_results_public.csv')

In [3]:
raw_data.shape

(51392, 154)

In [4]:
raw_data.dtypes

Respondent                int64
Professional             object
ProgramHobby             object
Country                  object
University               object
                         ...   
QuestionsInteresting     object
QuestionsConfusing       object
InterestedAnswers        object
Salary                  float64
ExpectedSalary          float64
Length: 154, dtype: object